#### Checking the GPU

In [1]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce GTX 1650 Ti (UUID: GPU-55b82302-073f-d276-910b-5e32debb81b0)


#### Imports

In [2]:
import torch
torch.cuda.empty_cache()

import time
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
import warnings
import pandas as pd

In [3]:
# Custom local imports
from src.dataloader import get_MELD_loaders
from src.parse import parse_opt
from src.trainer import build_model, trainer
from src.config import *

In [4]:
# To ignore the warnings from sklearn.metrics.classification_report
warnings.filterwarnings("ignore") 

#### Prepare the arguments and data

In [5]:
args = parse_opt()
args.cuda = not args.no_cuda and torch.cuda.is_available()

Args: {'batch_size': 2,
 'class_weight': False,
 'dir': './MELD_features/',
 'dropout': 0.2,
 'epochs': 100,
 'l2': 0.0003,
 'loss_fn': 'masked_nll',
 'lr': 0.0001,
 'mu': 0,
 'n_classes': 7,
 'no_cuda': False,
 'num_workers': 0,
 'seed': 100,
 'val_split': 0.25,
 'verbose': True}


In [6]:
model, optimizer, loss_function = build_model(D_m, D_q, D_g, D_r, D_e, D_h, args)

EmotionNet(
  (dropout): Dropout(p=0.2, inplace=False)
  (emo_rnn_b): EmotionRNN(
    (dropout): Dropout(p=0.2, inplace=False)
    (cell): EmotionGRUCell(
      (g_cell): GRUCell(600, 150)
      (p_cell): GRUCell(450, 150)
      (pl_cell): GRUCell(450, 150)
      (r_cell): GRUCell(450, 150)
      (rl_cell): GRUCell(450, 150)
      (e_cell): GRUCell(600, 150)
      (dropout): Dropout(p=0.2, inplace=False)
      (attention): SimpleAttention(
        (scalar): Linear(in_features=150, out_features=1, bias=False)
      )
    )
  )
  (emo_rnn_f): EmotionRNN(
    (dropout): Dropout(p=0.2, inplace=False)
    (cell): EmotionGRUCell(
      (g_cell): GRUCell(600, 150)
      (p_cell): GRUCell(450, 150)
      (pl_cell): GRUCell(450, 150)
      (r_cell): GRUCell(450, 150)
      (rl_cell): GRUCell(450, 150)
      (e_cell): GRUCell(600, 150)
      (dropout): Dropout(p=0.2, inplace=False)
      (attention): SimpleAttention(
        (scalar): Linear(in_features=150, out_features=1, bias=False)
      )
 

In [7]:
train_loader, valid_loader, test_loader = get_MELD_loaders(path = args.dir + 'MELD_features_raw.pkl',
                                                           n_classes=args.n_classes,
                                                           valid=args.val_split,
                                                           batch_size=args.batch_size,
                                                           num_workers=args.num_workers)

#### Training phase

In [8]:
model, metrics, best_label, best_pred, best_mask, best_attn, test_class_report = trainer(args, model, train_loader, valid_loader, test_loader, optimizer, loss_function)

Epoch [1]/[100]	 Train Loss: 1.662	 Train Acc: 41.960%	 Train F1: 35.470	 Val Loss: 1.615	 Val Acc: 47.710%	 Val F1: 31.670	 Test Loss: 1.606	 Test Acc: 49.120%	 Test F1: 34.320	 Time: 118.50 sec


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 4.00 GiB total capacity; 3.44 GiB already allocated; 0 bytes free; 3.46 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
train_losses = metrics['train']['train_losses']
train_fscores = metrics['train']['train_fscores']
train_accs = metrics['train']['train_accs']

val_losses = metrics['val']['val_losses']
val_fscores = metrics['val']['val_fscores']
val_accs = metrics['val']['val_accs']

test_losses = metrics['test']['test_losses']
test_fscores = metrics['test']['test_fscores']
test_accs = metrics['test']['test_accs']

best_fscore = metrics['best']['best_fscore']
best_loss = metrics['best']['best_loss']
best_acc = metrics['best']['best_acc']

NameError: name 'metrics' is not defined

#### Testing phase

In [ ]:
print('[Metrics]\t', end='')
print('Fscore: {:.3f}\tLoss: {:.3f}\tAccuracy: {:.3f}%\n'.format(best_fscore, best_loss, accuracy_score(best_label,best_pred,sample_weight=best_mask)*100))

In [ ]:
print('\nClassification report:')
print(classification_report(best_label,best_pred,sample_weight=best_mask,digits=4))

In [ ]:
print('\nConfusion matrix:')
print(confusion_matrix(best_label,best_pred,sample_weight=best_mask))

#### Save model

In [ ]:
model_name = 'models/EmoNet_'+str(int(round(best_fscore,0)))+'.pt'
torch.save(model.state_dict(), args.dir + model_name)
print('\nModel saved to {}.'.format(model_name))

In [ ]:
# Store all training metrics per epoch in a dataframe
train_metrics = pd.DataFrame(metrics['train'])
test_metrics = pd.DataFrame(metrics['test'])
val_metrics = pd.DataFrame(metrics['val'])
metrics = pd.concat([train_metrics, val_metrics, test_metrics], axis=1)
metrics.to_csv(args.dir + 'logs/metrics.csv', index=False)
print('Metrics saved to {}.'.format(args.dir + 'logs/metrics.csv'))